In [1]:
%%time
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dlib
import face_recognition
from sklearn.metrics import accuracy_score
import shutil
import warnings
warnings.filterwarnings('ignore')

from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import save_img
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

CPU times: total: 3.56 s
Wall time: 10.4 s


### face_locations: [(25, 80, 87, 17)]
### Top: The y-coordinate of the top edge of the bounding box.
### Right: The x-coordinate of the right edge of the bounding box.
### Bottom: The y-coordinate of the bottom edge of the bounding box.
### Left: The x-coordinate of the left edge of the bounding box.

In [ ]:
%%time
# Input and output directories
input_directory = r"C:\Users\mitvo\Documents\Facial_Image_Dataset\AffectNet_orginal_images(500)"
output_directory = r"C:\Users\mitvo\Documents\Facial_Image_Dataset\grayscaled_images(500)"

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Iterate through the directories in the input directory
for expression_dir in os.listdir(input_directory):
    expression_path = os.path.join(input_directory, expression_dir)
    if os.path.isdir(expression_path):
        output_expression_dir = os.path.join(output_directory, expression_dir)
        os.makedirs(output_expression_dir, exist_ok=True)
        
        # Iterate through images in the expression directory
        for image_file in os.listdir(expression_path):
            # Check if the file has a valid image extension
            if image_file.lower().endswith((".jpg", ".jpeg", ".png")):
                image_path = os.path.join(expression_path, image_file)
                rgb_image = cv2.imread(image_path)
                
                if rgb_image is not None:
                    grayscale_image = cv2.cvtColor(rgb_image,cv2.COLOR_BGR2GRAY)
                    new_file = image_file.rsplit(".",1)[0] + ".png"  # Change the output format to PNG
                    output_image_path = os.path.join(output_expression_dir, new_file)
                    cv2.imwrite(output_image_path, grayscale_image, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
                    output_image = cv2.imread(output_image_path)
#                     print(output_image.shape)
                else:
                    print(f"Failed to load image: {image_path}")
            else:
                print(f"Unsupported file format: {image_file}")

print("Grayscale conversion completed.")


In [ ]:
image=cv2.imread(r"C:\Users\mitvo\Documents\Facial_Image_Dataset\grayscaled_images(500)\NEUTRAL\ffhq_113.png")
image.shape

# For 500 images of affectnet dataset

In [ ]:
%%time
def load_labels_and_images(input_directory,output_directory):
    labels=[]
    for expression_index, expression_dir in enumerate(os.listdir(input_directory)):
        expression_path = os.path.join(input_directory, expression_dir)
        if os.path.isdir(expression_path):
            for image_file in os.listdir(expression_path):
                image_path = os.path.join(expression_path,image_file)
                image = cv2.imread(image_path)
                # Apply Gaussian blur
                img = cv2.GaussianBlur(image, ksize=(3,3),sigmaX=2)
                face_locations=face_recognition.face_locations(img)
                for i, face_location in enumerate(face_locations):
                    top,right,bottom,left = face_location # Extract coordinates
                    # Extract the face as a separate image
                    face=img[top:bottom, left:right]
                    cnvt_face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
                    output_label_dir = os.path.join(output_directory, str(expression_index))
                    os.makedirs(output_label_dir, exist_ok=True)
                    # Save the face portion as an image
                    output_image_path = os.path.join(output_label_dir,f"{i}_{image_file}")
                    save_img(output_image_path,cnvt_face)

In [ ]:
%%time
output_path=r"C:\Users\mitvo\Documents\Facial_Image_Dataset\face_recognised_cropped_images(500)"
train_dir_affect=r"C:\Users\mitvo\Documents\Facial_Image_Dataset\grayscaled_images(500)"
load_labels_and_images(train_dir_affect,output_path)

In [2]:
images_path=[]
input_directory=r"C:\Users\mitvo\Documents\Facial_Image_Dataset\face_recognised_cropped_images(500)"
for expression_index, expression_dir in enumerate(os.listdir(input_directory)):
    expression_path = os.path.join(input_directory, expression_dir)
    if os.path.isdir(expression_path):
        for image_file in os.listdir(expression_path):            
            image_path = os.path.join(expression_path,image_file)
            images_path.append(image_path)
print("Total face_recognised by face_recogniser on categorised_images :",len(images_path))

Total face_recognised by face_recogniser on categorised_images : 4091


In [3]:
cleaned_images=r"C:\Users\mitvo\Documents\Facial_Image_Dataset\face_recognised_cropped_images(500)"
image_shapes=[]
for expression_dir in os.listdir(cleaned_images):
    expression_path = os.path.join(cleaned_images,expression_dir)
    # Check if it's a directory
    if os.path.isdir(expression_path):
        # Iterate through images in the expression directory
        for image_file in os.listdir(expression_path):
            image_path = os.path.join(expression_path, image_file)
            image=cv2.imread(image_path)
            shape=image.shape
            image_shapes.append(shape)

In [4]:
from collections import Counter
Counter(image_shapes)

Counter({(62, 63, 3): 2516,
         (52, 52, 3): 710,
         (74, 75, 3): 557,
         (74, 74, 3): 112,
         (62, 62, 3): 78,
         (89, 90, 3): 38,
         (96, 96, 3): 23,
         (89, 86, 3): 22,
         (88, 96, 3): 16,
         (79, 90, 3): 10,
         (79, 86, 3): 3,
         (44, 43, 3): 3,
         (43, 43, 3): 2,
         (75, 74, 3): 1})

## Function to make the images of equal sizes i.e.,(62*63)

In [5]:
preprocessed_faces=[]
labels=[]
for expression_index, expression_dir in enumerate(os.listdir(cleaned_images)):
    expression_path = os.path.join(cleaned_images, expression_dir)
    if os.path.isdir(expression_path):
        for image_file in os.listdir(expression_path):
            image_path = os.path.join(expression_path, image_file)
            image = cv2.imread(image_path)
            preprocessed_faces.append(image)
            labels.append(expression_index)

In [6]:
preprocessed_faces

[array([[[25, 25, 25],
         [23, 23, 23],
         [20, 20, 20],
         ...,
         [20, 20, 20],
         [19, 19, 19],
         [19, 19, 19]],
 
        [[22, 22, 22],
         [18, 18, 18],
         [15, 15, 15],
         ...,
         [18, 18, 18],
         [18, 18, 18],
         [18, 18, 18]],
 
        [[17, 17, 17],
         [13, 13, 13],
         [ 9,  9,  9],
         ...,
         [17, 17, 17],
         [17, 17, 17],
         [17, 17, 17]],
 
        ...,
 
        [[24, 24, 24],
         [29, 29, 29],
         [33, 33, 33],
         ...,
         [14, 14, 14],
         [13, 13, 13],
         [12, 12, 12]],
 
        [[20, 20, 20],
         [26, 26, 26],
         [30, 30, 30],
         ...,
         [14, 14, 14],
         [12, 12, 12],
         [ 9,  9,  9]],
 
        [[14, 14, 14],
         [23, 23, 23],
         [28, 28, 28],
         ...,
         [11, 11, 11],
         [ 9,  9,  9],
         [ 8,  8,  8]]], dtype=uint8),
 array([[[  0,   0,   0],
         [  1,  

In [7]:
target_size = (62,63,3)
processed_faces = []

# Assuming preprocessed_faces contains images of varying sizes
for image in preprocessed_faces:
    if image.shape[0] < target_size[0] or image.shape[1] < target_size[1]:
        scaling_factor = min(target_size[0] / image.shape[0], target_size[1] / image.shape[1])
        new_width=int(image.shape[1]*scaling_factor)
        new_height=int(image.shape[0]*scaling_factor)
        resized_image=cv2.resize(image,(new_width,new_height),interpolation=cv2.INTER_LANCZOS4)
        # Creating a blank image of the target size with the same number of channels as the original image
        blank_image = np.zeros((target_size[0],target_size[1],image.shape[2]), dtype=np.uint8)
        # Calculating the position to paste the resized image in the blank image to center it
        x_offset = (target_size[1] - resized_image.shape[1])//2
        y_offset = (target_size[0] - resized_image.shape[0])//2
         # Pasting the resized image into the blank image
        blank_image[y_offset:y_offset + resized_image.shape[0],x_offset:x_offset + resized_image.shape[1], :] = resized_image
        processed_faces.append(blank_image) 
    else:
        cropped_image=cv2.resize(image,(target_size[1],target_size[0]))
        processed_faces.append(cropped_image)

In [8]:
len(processed_faces) #length of preprocessed faces in the form of pixels

4091

In [9]:
shapes=[]
for image in processed_faces:
    shapes.append(image.shape)
    
Counter(shapes)

Counter({(62, 63, 3): 4091})

In [10]:
len(labels)

4091

In [11]:
processed_faces[10].shape

(62, 63, 3)

In [12]:
pixel_data = np.array(processed_faces)

datagen = ImageDataGenerator(horizontal_flip=True,zoom_range=0.2,
                             shear_range=0.2,featurewise_center=True,
                             featurewise_std_normalization=True )
datagen.fit(pixel_data)

generator = datagen.flow(x=pixel_data, batch_size=len(processed_faces),shuffle=False)

preprocessed_data = next(generator)

In [13]:
preprocessed_data.shape

(4091, 62, 63, 3)

In [14]:
pixel_data.shape

(4091, 62, 63, 3)

In [15]:
#pip install optuna

In [16]:
import optuna
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense
import tensorflow as tf

def create_cnn_model(optimizer_name, learning_rate, dropout_rate, l2_penalty):
    model = Sequential()
    
    model.add(Conv2D(32, (2,2), activation='relu', input_shape=(62, 63, 3), kernel_regularizer=tf.keras.regularizers.l2(l2_penalty)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (2, 2), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))

    model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_penalty)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())

    model.add(Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_penalty)))
    model.add(Dropout(dropout_rate))

    model.add(Dense(8, activation='softmax'))

    if optimizer_name == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer_name == "rmsprop":
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        raise ValueError("Invalid optimizer name")

    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

    return model


In [18]:
num_classes = len(np.unique(labels))

In [19]:
np.unique(labels)

array([0, 1, 2, 3, 4, 5, 6, 7])

# Training and validating the pixel data using Stratified Kfold method.

In [20]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping

def objective(trial):
    dropout_rate = trial.suggest_float("dropout_rate", 0, 0.25)
    epochs = trial.suggest_int("epochs",3,10)
    optimizer_name = trial.suggest_categorical("optimizer_name", ["adam","sgd","rmsprop"])
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-2, log=True)
    l2_penalty = trial.suggest_float("l2_penalty", 1e-6, 1e-2, log=True)
    num_fold = 5

    # Store other metrics in addition to accuracy and evaluate the model based on these metrics
    val_losses = []
    val_accuracies = []

    cross_validator = StratifiedKFold(n_splits=num_fold, shuffle=True, random_state=42)

    for fold, (train_indices, val_indices) in enumerate(cross_validator.split(preprocessed_data, Labels)):
        X_train_fold, X_val_fold = preprocessed_data[train_indices], preprocessed_data[val_indices]
        y_train_fold, y_val_fold = Labels[train_indices], Labels[val_indices]

        model = create_cnn_model(optimizer_name=optimizer_name, learning_rate=learning_rate, dropout_rate=dropout_rate, l2_penalty=l2_penalty)

        X_train_tf = tf.convert_to_tensor(X_train_fold)
        y_train_tf = tf.keras.utils.to_categorical(y_train_fold, num_classes=8)

        X_val_tf = tf.convert_to_tensor(X_val_fold)
        y_val_tf = tf.keras.utils.to_categorical(y_val_fold, num_classes=8)

        early_stopping = EarlyStopping(monitor='val_loss', patience=8, mode='min', verbose=1)
        history = model.fit(X_train_tf, y_train_tf, epochs=epochs, validation_data=(X_val_tf, y_val_tf), callbacks=[early_stopping], verbose=2)
        
        val_accuracies.append(history.history["val_accuracy"][-1])
        val_losses.append(history.history["val_loss"][-1])

    # Calculate the average of validation accuracy and loss across folds
    avg_val_accuracy = np.mean(val_accuracies, axis=0)
    avg_val_loss = np.mean(val_losses, axis=0) 
    print(avg_val_loss)
    # You can use these metrics to guide the optimization process
    return -avg_val_accuracy  # Minimize negative of accuracy, or other adjustments as needed


In [ ]:
import optuna
import threading
import tensorflow as tf
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping

# Global variables to store the best parameters
best_params = {}
best_loss = float('inf')

# Custom StratifiedKFold splitter to ensure equal samples from each class
class EqualClassStratifiedKFold(StratifiedKFold):
    def split(self, X, y, groups=None):
        unique_classes, class_counts = np.unique(y, return_counts=True)
        min_samples = np.min(class_counts)  # Determine the minimum number of samples in a class

        # Select equal samples from each class
        idx_equal = []
        for c in unique_classes:
            class_indices = np.where(y == c)[0][:min_samples]
            idx_equal.extend(class_indices)
            
        idx_equal = np.random.permutation(idx_equal)  # Shuffle the indices to ensure randomness

        # Use the same indices for each fold
        for x in range(self.n_splits):
            yield np.arange(len(X)), idx_equal

def create_cnn_model(optimizer_name, learning_rate, dropout_rate, l2_penalty):
    # Implement your CNN model creation here
    pass

def objective(trial):
    global best_params, best_loss

    dropout_rate = trial.suggest_float("dropout_rate", 0, 0.25)
    epochs = trial.suggest_int("epochs", 3, 10)
    optimizer_name = trial.suggest_categorical("optimizer_name", ["adam", "sgd", "rmsprop"])
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-2, log=True)
    l2_penalty = trial.suggest_float("l2_penalty", 1e-6, 1e-2, log=True)
    num_fold = 5

    # Store other metrics in addition to accuracy and evaluate the model based on these metrics
    val_losses = []
    val_accuracies = []

    cross_validator = EqualClassStratifiedKFold(n_splits=num_fold, shuffle=True, random_state=42)

    for fold, (train_indices, val_indices) in enumerate(cross_validator.split(preprocessed_data, Labels)):
        X_train_fold, X_val_fold = preprocessed_data[train_indices], preprocessed_data[val_indices]
        y_train_fold, y_val_fold = Labels[train_indices], Labels[val_indices]

        model = create_cnn_model(optimizer_name=optimizer_name, learning_rate=learning_rate, dropout_rate=dropout_rate, l2_penalty=l2_penalty)

        X_train_tf = tf.convert_to_tensor(X_train_fold)
        y_train_tf = tf.keras.utils.to_categorical(y_train_fold, num_classes=8)

        X_val_tf = tf.convert_to_tensor(X_val_fold)
        y_val_tf = tf.keras.utils.to_categorical(y_val_fold, num_classes=8)

        early_stopping = EarlyStopping(monitor='val_loss', patience=8, mode='min', verbose=1)
        history = model.fit(X_train_tf, y_train_tf, epochs=epochs, validation_data=(X_val_tf, y_val_tf), callbacks=[early_stopping], verbose=2)
        
        val_accuracies.append(history.history["val_accuracy"][-1])
        val_losses.append(history.history["val_loss"][-1])

    # Calculate the average of validation accuracy and loss across folds
    avg_val_accuracy = np.mean(val_accuracies, axis=0)
    avg_val_loss = np.mean(val_losses, axis=0) 

    # Update best parameters if current trial is better
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        best_params = {
            'dropout_rate': dropout_rate,
            'epochs': epochs,
            'optimizer_name': optimizer_name,
            'learning_rate': learning_rate,
            'l2_penalty': l2_penalty
        }

    return avg_val_loss

def optimize_with_threading():
    study = optuna.create_study(direction='minimize')
    num_trials = 10

    def optimize_trial(trial):
        study.optimize(objective, n_trials=num_trials)

    threads = []
    for _ in range(5):  # Number of threads
        thread = threading.Thread(target=optimize_trial, args=(study,))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial
    print("  Value: ", trial.value)
    print("  Params: ", best_params)

if __name__ == "__main__":
    optimize_with_threading()

In [21]:
labels =list(map(int,labels))
Labels=np.array(labels)

In [ ]:
%%time
study = optuna.create_study(study_name = "Model training to detect facial emotion recognition",direction = "minimize")
study.optimize(objective,n_trials=15)

[I 2023-11-13 10:25:04,977] A new study created in memory with name: Model training to detect facial emotion recognition


Epoch 1/5
103/103 - 13s - loss: 2.4680 - accuracy: 0.2518 - val_loss: 2.2973 - val_accuracy: 0.1551 - 13s/epoch - 126ms/step
Epoch 2/5
103/103 - 11s - loss: 1.7868 - accuracy: 0.4187 - val_loss: 2.3882 - val_accuracy: 0.1746 - 11s/epoch - 103ms/step
Epoch 3/5
103/103 - 10s - loss: 1.5226 - accuracy: 0.5040 - val_loss: 2.2629 - val_accuracy: 0.2100 - 10s/epoch - 98ms/step
Epoch 4/5
103/103 - 10s - loss: 1.3248 - accuracy: 0.5981 - val_loss: 2.0127 - val_accuracy: 0.3162 - 10s/epoch - 98ms/step
Epoch 5/5
103/103 - 11s - loss: 1.1700 - accuracy: 0.6458 - val_loss: 1.8287 - val_accuracy: 0.3919 - 11s/epoch - 107ms/step
Epoch 1/5
103/103 - 12s - loss: 2.3705 - accuracy: 0.2756 - val_loss: 2.1855 - val_accuracy: 0.1736 - 12s/epoch - 121ms/step
Epoch 2/5
103/103 - 11s - loss: 1.7421 - accuracy: 0.4280 - val_loss: 2.1292 - val_accuracy: 0.2274 - 11s/epoch - 110ms/step
Epoch 3/5
103/103 - 11s - loss: 1.4834 - accuracy: 0.5197 - val_loss: 1.9969 - val_accuracy: 0.3166 - 11s/epoch - 107ms/step
Ep

[I 2023-11-13 10:29:59,269] Trial 0 finished with value: -0.39085772037506106 and parameters: {'dropout_rate': 0.06055052720341078, 'epochs': 5, 'optimizer_name': 'adam', 'learning_rate': 5.4664340074036264e-05, 'l2_penalty': 0.00015088188452035397}. Best is trial 0 with value: -0.39085772037506106.


1.864879584312439
Epoch 1/6
103/103 - 12s - loss: 4.8402 - accuracy: 0.2417 - val_loss: 6.9127 - val_accuracy: 0.1306 - 12s/epoch - 114ms/step
Epoch 2/6
103/103 - 11s - loss: 2.6831 - accuracy: 0.3484 - val_loss: 8.4066 - val_accuracy: 0.1306 - 11s/epoch - 102ms/step
Epoch 3/6
103/103 - 10s - loss: 2.2254 - accuracy: 0.4230 - val_loss: 4.8964 - val_accuracy: 0.1783 - 10s/epoch - 100ms/step
Epoch 4/6
103/103 - 11s - loss: 1.9374 - accuracy: 0.5122 - val_loss: 2.6560 - val_accuracy: 0.3211 - 11s/epoch - 110ms/step
Epoch 5/6
103/103 - 11s - loss: 1.7459 - accuracy: 0.5837 - val_loss: 2.3516 - val_accuracy: 0.4359 - 11s/epoch - 103ms/step
Epoch 6/6
103/103 - 11s - loss: 1.5672 - accuracy: 0.6632 - val_loss: 2.5139 - val_accuracy: 0.4310 - 11s/epoch - 109ms/step
Epoch 1/6
103/103 - 12s - loss: 5.4711 - accuracy: 0.2441 - val_loss: 9.1790 - val_accuracy: 0.1137 - 12s/epoch - 115ms/step
Epoch 2/6
103/103 - 10s - loss: 2.7177 - accuracy: 0.3492 - val_loss: 5.2899 - val_accuracy: 0.1822 - 10s/e

[I 2023-11-13 10:35:42,649] Trial 1 finished with value: -0.44952787160873414 and parameters: {'dropout_rate': 0.14861492882278593, 'epochs': 6, 'optimizer_name': 'rmsprop', 'learning_rate': 0.0011780805377883086, 'l2_penalty': 0.001795537450144677}. Best is trial 1 with value: -0.44952787160873414.


...

2.6448811054229737
Epoch 1/4
103/103 - 12s - loss: 3.7787 - accuracy: 0.2402 - val_loss: 9.1459 - val_accuracy: 0.1148 - 12s/epoch - 117ms/step
Epoch 2/4
103/103 - 11s - loss: 1.7891 - accuracy: 0.3744 - val_loss: 8.3925 - val_accuracy: 0.1453 - 11s/epoch - 106ms/step
Epoch 3/4
103/103 - 11s - loss: 1.4872 - accuracy: 0.4719 - val_loss: 2.2033 - val_accuracy: 0.3724 - 11s/epoch - 105ms/step
Epoch 4/4
103/103 - 11s - loss: 1.2339 - accuracy: 0.5639 - val_loss: 2.3516 - val_accuracy: 0.3590 - 11s/epoch - 106ms/step
Epoch 1/4
103/103 - 12s - loss: 3.6135 - accuracy: 0.2515 - val_loss: 6.0564 - val_accuracy: 0.1748 - 12s/epoch - 112ms/step
Epoch 2/4
103/103 - 11s - loss: 1.7496 - accuracy: 0.3767 - val_loss: 7.6173 - val_accuracy: 0.1760 - 11s/epoch - 105ms/step
Epoch 3/4
103/103 - 11s - loss: 1.4665 - accuracy: 0.4849 - val_loss: 2.8552 - val_accuracy: 0.3154 - 11s/epoch - 107ms/step
Epoch 4/4
103/103 - 11s - loss: 1.2199 - accuracy: 0.5750 - val_loss: 1.9421 - val_accuracy: 0.3912 - 11s/

[I 2023-11-13 10:58:17,168] Trial 6 finished with value: -0.3803523302078247 and parameters: {'dropout_rate': 0.08310428440875858, 'epochs': 4, 'optimizer_name': 'rmsprop', 'learning_rate': 0.0013665640110120617, 'l2_penalty': 4.702691037787278e-06}. Best is trial 1 with value: -0.44952787160873414.


2.080736756324768
Epoch 1/8
103/103 - 12s - loss: 4.0092 - accuracy: 0.1357 - val_loss: 2.1334 - val_accuracy: 0.1319 - 12s/epoch - 120ms/step
Epoch 2/8
103/103 - 11s - loss: 3.8395 - accuracy: 0.1357 - val_loss: 2.1691 - val_accuracy: 0.1538 - 11s/epoch - 110ms/step
Epoch 3/8
103/103 - 11s - loss: 3.7333 - accuracy: 0.1375 - val_loss: 2.1837 - val_accuracy: 0.1282 - 11s/epoch - 108ms/step
Epoch 4/8
103/103 - 11s - loss: 3.6394 - accuracy: 0.1369 - val_loss: 2.2224 - val_accuracy: 0.1368 - 11s/epoch - 105ms/step
Epoch 5/8
103/103 - 12s - loss: 3.5088 - accuracy: 0.1363 - val_loss: 2.2863 - val_accuracy: 0.1441 - 12s/epoch - 118ms/step
Epoch 6/8
103/103 - 12s - loss: 3.4846 - accuracy: 0.1293 - val_loss: 2.3471 - val_accuracy: 0.1380 - 12s/epoch - 117ms/step
Epoch 7/8
103/103 - 12s - loss: 3.4570 - accuracy: 0.1354 - val_loss: 2.3884 - val_accuracy: 0.1380 - 12s/epoch - 113ms/step
Epoch 8/8
103/103 - 11s - loss: 3.3587 - accuracy: 0.1381 - val_loss: 2.4040 - val_accuracy: 0.1355 - 11s/e

[I 2023-11-13 11:06:28,410] Trial 7 finished with value: -0.14739963114261628 and parameters: {'dropout_rate': 0.2297662178022916, 'epochs': 8, 'optimizer_name': 'sgd', 'learning_rate': 8.821558425209735e-06, 'l2_penalty': 1.394159684873311e-05}. Best is trial 1 with value: -0.44952787160873414.


2.4430596828460693
Epoch 1/6
103/103 - 15s - loss: 6.6474 - accuracy: 0.2179 - val_loss: 3.0007 - val_accuracy: 0.2271 - 15s/epoch - 148ms/step
Epoch 2/6
103/103 - 13s - loss: 2.5966 - accuracy: 0.3227 - val_loss: 2.8544 - val_accuracy: 0.2796 - 13s/epoch - 129ms/step
Epoch 3/6
103/103 - 14s - loss: 2.3474 - accuracy: 0.3454 - val_loss: 2.4297 - val_accuracy: 0.3223 - 14s/epoch - 131ms/step
Epoch 4/6
103/103 - 14s - loss: 2.1665 - accuracy: 0.3826 - val_loss: 2.7133 - val_accuracy: 0.2955 - 14s/epoch - 134ms/step
Epoch 5/6
103/103 - 14s - loss: 2.0923 - accuracy: 0.3894 - val_loss: 2.1645 - val_accuracy: 0.3639 - 14s/epoch - 133ms/step
Epoch 6/6


In [ ]:
best_params1_cnn2 = study.best_params
best_params1_cnn2

In [ ]:
import json

with open('best_params.json', 'w') as f:
    json.dump(best_params1_cnn2, f)

In [ ]:
%%time
num_folds = 5
best_model = None
best_train_metric = 0.99
best_valid_metric = 0.98

best_params = study.best_params
#Define the hyperparameters (best_optimiser, best_learning_rate, best_drop_out)
best_optimiser = best_params["optimizer_name"] 
best_learning_rate = best_params["learning_rate"] 
best_drop_out = best_params['dropout_rate']
best_epochs = best_params["epochs"]
best_l2_penality=best_params["l2_penalty"]
# Create and compile your ResNet model (similar to your create_resnet_model function)
model = create_cnn_model(optimizer_name=best_optimiser,learning_rate=best_learning_rate,dropout_rate=best_drop_out,l2_penalty=best_l2_penality)

# Define the cross-validator (StratifiedKFold or KFold based on your data)
cross_validator = StratifiedKFold(n_splits=num_folds,shuffle=True,random_state=42)

# Iterate through the folds
for fold, (train_indices, val_indices) in enumerate(cross_validator.split(preprocessed_data,Labels)):
    # Split the data into training and validation sets for this fold
    X_train_fold, X_val_fold = preprocessed_data[train_indices],preprocessed_data[val_indices]
    y_train_fold, y_val_fold = Labels[train_indices],Labels[val_indices]

    # Converting numpy arrays to TensorFlow tensors
    X_train_tf = tf.convert_to_tensor(X_train_fold)
    y_train_tf = tf.keras.utils.to_categorical(y_train_fold,num_classes=8)

    X_val_tf = tf.convert_to_tensor(X_val_fold)
    y_val_tf = tf.keras.utils.to_categorical(y_val_fold,num_classes=8)
    
    # Train the model on this fold
    history = model.fit(X_train_tf, y_train_tf,epochs=best_epochs,verbose=2,validation_data=(X_val_tf,y_val_tf))
    val_metric = model.evaluate(X_val_tf,y_val_tf)[1] 
    train_metric=model.evaluate(X_train_tf,y_train_tf)[1]
    # Check if the current model is the best so far
    if val_metric > best_valid_metric and train_metric > best_train_metric:
        best_metric = val_metric
        best_model = model

# Saving the best model to disk
best_model.save('Face_emotion_recognition_trail6_4091_IMAGES_LANCOSZ_cnn2.h5')